# Spectrogram classification 모델 구현
> **2차원 Spectrogram 데이터를 입력받아 위 모델과 동일한 역할을 수행하는 모델을 만들어보기.**
***
> **1. 데이터 처리와 분류**
> - **라벨 데이터 처리하기**
> - **sklearn의 train_test_split함수를 이용하여 train, test 분리**  
>  
> **2. 학습을 위한 하이퍼파라미터 설정**  
>  
> **3. 데이터셋 구성**  
> - **tf.data.Dataset을 이용**
> - **from_tensor_slices 함수에 return 받길 원하는 데이터를 튜플 (data, label) 형태로 넣어서 사용**  
> - **map과 batch를 이용한 데이터 전처리**  
> - **주의 : waveform을 spectrogram으로 변환하기 위해 추가로 사용하는 
> 메모리 때문에 이후 메모리 부족 현상을 겪게 될수도 있습니다.**  
> - **tf.data.Dataset이 생성된 이후, 아래 예시와 같이 wav 데이터나 
> spectrogram 데이터를 담아둔 메모리 버퍼를 비워 주면 도움이 됩니다.**
>  
> **4. 2차원 Spectrogram 데이터를 처리하는 모델 구성**
> - **2차원 Spectrogram 데이터의 시간축 방향으로 Conv1D layer를 적용, 혹은 Conv2D layer를 적용 가능**  
> - **batchnorm, dropout, dense layer 등을 이용**
> - **12개의 단어 class를 구분하는 loss를 사용하고 Adam optimizer를 사용**
> - **모델 가중치를 저장하는 checkpoint callback 함수 추가**  
> - **다양한 모델의 실험을 진행해 보시기 바랍니다.**
>  
> **5. 학습 후, 학습이 어떻게 진행됐는지 그래프로 출력**
> - **loss, accuracy를 그래프로 표현**
>  
> **6. Test dataset을 이용해서 모델의 성능을 평가**
> - **저장한 weight 불러오기**
> - **모델의 예측값과 정답값이 얼마나 일치하는지 확인**


In [8]:
# 1. 데이터 처리와 분류
# 1.1 데이터 셋 가져오기(waveform형태의 데이터)
import numpy as np
import os
data_path = os.getenv("HOME")+'/aiffel/speech_recognition/data/speech_wav_8000.npz'
speech_data = np.load(data_path)

In [42]:
# 1.2 데이터 확인
import IPython.display as ipd
import random

# 데이터 선택 (랜덤하게 선택하고 있으니, 여러번 실행해 보기)
rand = random.randint(0, len(speech_data["wav_vals"]))
print("rand num : ", rand)

sr = 8000 # 1초동안 재생되는 샘플의 갯수
data = speech_data["wav_vals"][rand]
print("Wave data shape : ", data.shape)
print("label : ", speech_data["label_vals"][rand])

ipd.Audio(data, rate=sr)

rand num :  33808
Wave data shape :  (8000,)
label :  ['on']
나온다
[[-1.27418665e-04 -1.12644804e-04 -1.86756923e-04 ... -1.62762426e-05
  -4.93293861e-04 -3.55132594e-04]
 [-3.06998868e-03 -4.10807552e-03 -3.31036211e-03 ... -4.67803935e-03
  -4.21488844e-03 -6.10053446e-03]
 [ 1.59116389e-04  1.11387717e-03 -1.25855627e-03 ... -9.50679742e-03
   1.47810066e-02  3.41885872e-02]
 ...
 [-7.13879392e-02  1.11577094e-01 -3.49124521e-01 ...  1.27255514e-01
  -2.06745356e-01  2.13672481e-02]
 [ 2.80666262e-01  1.76843733e-01 -8.22633058e-02 ...  4.30828370e-02
   8.51763189e-02 -1.32893130e-01]
 [-5.42262085e-02  1.66265428e-01  3.59905154e-01 ... -3.49392533e-01
   2.62163337e-02  2.09347770e-01]]


In [37]:
# 라벨처리
target_list = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

label_value = target_list
label_value.append('unknown')
label_value.append('silence')

print('LABEL : ', label_value)

new_label_value = dict()
for i, l in enumerate(label_value):
    new_label_value[l] = i
label_value = new_label_value

print('Indexed LABEL : ', new_label_value)

LABEL :  ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', 'silence']
Indexed LABEL :  {'yes': 0, 'no': 1, 'up': 2, 'down': 3, 'left': 4, 'right': 5, 'on': 6, 'off': 7, 'stop': 8, 'go': 9, 'unknown': 10, 'silence': 11}


In [1]:
import librosa

def wav2spec(wav, fft_size=258): # spectrogram shape을 맞추기위해서 size 변형
    D = np.abs(librosa.stft(wav, n_fft=fft_size))
    return D

In [45]:
# 위에서 뽑았던 sample data를 spectrogram으로 변경
spec = wav2spec(data)
print("Waveform shape : ",data.shape)
print("Spectrogram shape : ",spec.shape)

Waveform shape :  (8000,)
Spectrogram shape :  (130, 126)


In [40]:
# 학습을 위한 데이터 분리
from sklearn.model_selection import train_test_split

sr = 8000 # 1초동안 재생되는 샘플의 갯수
train_wav, test_wav, train_label, test_label = train_test_split(speech_data["wav_vals"], 
                                                                label_data, 
                                                                test_size=0.1,
                                                                shuffle=True)
print(train_wav)

train_wav = train_wav.reshape([-1, sr, 1]) # add channel for CNN
test_wav = test_wav.reshape([-1, sr, 1])

[[-5.5711297e-04 -7.7047281e-04 -5.1892240e-04 ... -1.5258178e-03
  -1.5083402e-03 -1.3132961e-03]
 [ 1.0443663e-05  1.4051682e-05  2.9038580e-05 ...  3.1192670e-07
  -2.4640449e-05 -7.2985960e-05]
 [-9.7145425e-04 -6.6215120e-04  3.7479235e-04 ...  2.7470032e-03
   2.7592566e-03  2.2672398e-03]
 ...
 [ 3.3109358e-03  4.7446517e-03  7.6106415e-05 ...  2.6764479e-03
   2.1537682e-03 -1.8746440e-03]
 [ 6.0911132e-03  1.0702068e-02  7.2187753e-03 ... -3.5751252e-03
   6.4412719e-03  6.7144413e-03]
 [ 3.7578691e-03  5.8623916e-03  1.2093403e-03 ...  1.0265530e-03
   2.2018794e-06 -3.3511594e-03]]


In [ ]:
# 2. 학습을 위한 하이퍼파라미터 설정
batch_size = 32
max_epochs = 10

# the save point
checkpoint_dir = os.getenv('HOME')+'/aiffel/speech_recognition/models/spec'

checkpoint_dir

In [39]:
# 3. 데이터셋 구성
def one_hot_label(spec, label):
    # tf.one_hot은 범주형 데이터에 사용하며
    # 주어진 리스트를 답지로 반환해준다.
    # ex) list[red, green, blue]일 경우
    # tf.one_hot([red,green,blue],3)은
    # [1,0,0],[0,1,0],[0,0,1]이 된다.
    label = tf.one_hot(label, depth=12)
    return spec, label

# tf.data.Dataset을 이용
#from_tensor_slices 함수에 return 받길 원하는 데이터를 튜플 (data, label) 형태로 넣어서 사용
# map과 batch를 이용한 데이터 전처리
#주의 : waveform을 spectrogram으로 변환하기 위해 추가로 사용하는 메모리 때문에 이후 메모리 부족 현상을 겪게 될수도 있습니다.
#tf.data.Dataset이 생성된 이후, 아래 예시와 같이 wav 데이터나 spectrogram 데이터를 담아둔 메모리 버퍼를 비워 주면 도움이 됩니다.

import tensorflow as tf

# for train
train_dataset = tf.data.Dataset.from_tensor_slices((train_wav, train_label))
train_dataset = train_dataset.map(one_hot_label)
train_dataset = train_dataset.repeat().batch(batch_size=batch_size)
print(train_dataset)

# for test
test_dataset = tf.data.Dataset.from_tensor_slices((test_wav, test_label))
test_dataset = test_dataset.map(one_hot_label)
test_dataset = test_dataset.batch(batch_size=batch_size)
print(test_dataset)

NameError: name 'train_wav' is not defined

In [ ]:
# 4. 2차원 Spectrogram 데이터를 처리하는 모델 구성

In [ ]:

2차원 Spectrogram 데이터의 시간축 방향으로 Conv1D layer를 적용, 혹은 Conv2D layer를 적용 가능
batchnorm, dropout, dense layer 등을 이용
12개의 단어 class를 구분하는 loss를 사용하고 Adam optimizer를 사용
모델 가중치를 저장하는 checkpoint callback 함수 추가
다양한 모델의 실험을 진행해 보시기 바랍니다.
5. 학습 후, 학습이 어떻게 진행됐는지 그래프로 출력

loss, accuracy를 그래프로 표현
6. Test dataset을 이용해서 모델의 성능을 평가

저장한 weight 불러오기
모델의 예측값과 정답값이 얼마나 일치하는지 확인